# 利用VGG/GoogleNet/ResNet 等结构对CIFAR-10进行分类

导入相关包

In [1]:
import torch
import torch.nn.functional as F
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR

定义是否使用GPU

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

数据准备，尝试利用数据增强和归一化，
提示：transforms.RandomCrop(),transforms.RandomHorizontalFlip(),
transforms.Normalize()等函数
可以设计函数获取数据的均值和方差（可选）

In [3]:
def get_mean_and_std(dataset):
    '''
        可以设计函数获取数据的均值和方差
    '''
    # 初始化数据均值和方差
    mean = [0, 0, 0]
    std = [0, 0, 0]
    
    # 获取数据集的图片数量
    imgs_num = len(dataset)
    
    # 遍历数据集的张量和标签
    for img, _ in dataset:
        for i in range(3): # 遍历图片的RGB三通道
            # 计算每个通道的均值和标准差
            mean[i] += torch.mean(img[i, :, :])
            std[i] += torch.std(img[i, :, :])
            
    data_mean = [mean[i] / imgs_num for i in range(3)]
    data_std = [std[i] / imgs_num for i in range(3)]
    return data_mean, data_std

# 准备数据集
trainset = tv.datasets.CIFAR10(root='/data/cifar10', train=True, download=True, 
                               transform=transforms.ToTensor())
testset = tv.datasets.CIFAR10(root='/data/cifar10', train=False, download=True,
                              transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=get_mean_and_std(trainset)[0], std=get_mean_and_std(trainset)[1])
])
print("trainset: ", get_mean_and_std(trainset))

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=get_mean_and_std(testset)[0], std=get_mean_and_std(testset)[1])
])
print("testset: ", get_mean_and_std(testset))

trainset:  ([tensor(0.4914), tensor(0.4822), tensor(0.4465)], [tensor(0.2023), tensor(0.1994), tensor(0.2010)])
testset:  ([tensor(0.4942), tensor(0.4851), tensor(0.4504)], [tensor(0.2020), tensor(0.1991), tensor(0.2011)])


In [5]:
Batch_size = 64

trainset = tv.datasets.CIFAR10(root='/data/cifar10', train=True, download=True,
                               transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=Batch_size, shuffle=True, num_workers=0)

testset = tv.datasets.CIFAR10(root='/data/cifar10', train=False, download=True,
                              transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=Batch_size, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


定义网络结构，选取VGG/googlenet/resnet 等网络结构

In [6]:
# 定义残差块Resnet_block
class Resnet_block(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1, res=True):
        # 调用父类的构造函数
        super(Resnet_block, self).__init__()
        
        self.res = res # 是否带残差连接
        
        self.left = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        
        if stride !=1 or in_channels!=out_channels:
            self.short = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.short = None
            
        self.relu = nn.Sequential(
            nn.ReLU(inplace=True)
        )
            
    def forward(self, x):
        out = self.left(x)
        if self.res:
            out += self.short(x)
        out = self.relu(out)
        return out

In [7]:
class YourNet(nn.Module):
    '''
        设计卷积神经网络，本作业中使用VGG
    '''
    def __init__(self, cfg=[64, 'M', 128, 'M', 256, 'M', 512, 'M'], res=True):
        
        # 调用父类的构造函数
        super(YourNet, self).__init__()
    
        self.res = res # 是否带残差连接
        self.cfg = cfg # 配置列表
        self.in_channels = 3 # 输入通道数
        self.futures = self.create_layer()
        
        self.classifier = nn.Sequential(nn.Dropout(0.4), nn.Linear(4*512, 10), )
        
    def create_layer(self):
        layers = []
        for i in self.cfg:
            if i == 'M':
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            else:
                layers.append(Resnet_block(self.in_channels, i, self.res))
                self.in_channels = i    # 输入通道数改为上一层的输出通道数    
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.futures(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

实例化网络，尝试加载保存过的模型继续训练，torch.load()函数

In [8]:
# 实例化网络
model = YourNet().to(device)
model_path = 'cifar10_data.pth'
torch.save(model.state_dict(), model_path)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

超参数设置，定义损失函数和优化方式

In [9]:
# 超参数设置
LR = 0.01
start_epoch = 0

# 损失函数和优化方式
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=LR, weight_decay=5e-4)

# 改变学习率
scheduler = MultiStepLR(optimizer, milestones=[20, 40], gamma=0.1)

## 训练

In [10]:
def train(epoch):
    '''
        Your code
    '''
    sum_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
            
        # 梯度清零
        optimizer.zero_grad()
            
        # 前向传播，计算损失，反向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
            
        # 更新参数
        optimizer.step()
            
        sum_loss += loss.item()

    print('第{0}次训练 loss:{1:.6f}'.format(epoch+1, sum_loss/100))
    
    # 阶段性更新学习率
    scheduler.step()

## 测试
    

In [11]:
def test(epoch):
    '''
        注意：保存当前训练最优模型
    '''
    # 切换模型为评估模式
    model.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print('第{}次训练的准确率为:{}%'.format(epoch+1, 100*correct/total))

In [12]:
# 训练、测试

# 经过70次训练，准确率可达90%以上
for epoch in range(start_epoch, start_epoch+70):
    train(epoch)
    test(epoch)

第1次训练 loss:13.746835
第1次训练的准确率为:59.53%
第2次训练 loss:8.368948
第2次训练的准确率为:68.14%
第3次训练 loss:6.511558
第3次训练的准确率为:72.23%
第4次训练 loss:5.570038
第4次训练的准确率为:72.51%
第5次训练 loss:4.978410
第5次训练的准确率为:79.12%
第6次训练 loss:4.502523
第6次训练的准确率为:81.3%
第7次训练 loss:4.162971
第7次训练的准确率为:81.5%
第8次训练 loss:3.898629
第8次训练的准确率为:82.83%
第9次训练 loss:3.659835
第9次训练的准确率为:84.72%
第10次训练 loss:3.465477
第10次训练的准确率为:83.25%
第11次训练 loss:3.274351
第11次训练的准确率为:85.19%
第12次训练 loss:3.146001
第12次训练的准确率为:83.78%
第13次训练 loss:2.997351
第13次训练的准确率为:85.03%
第14次训练 loss:2.890667
第14次训练的准确率为:85.35%
第15次训练 loss:2.733374
第15次训练的准确率为:86.04%
第16次训练 loss:2.701761
第16次训练的准确率为:86.48%
第17次训练 loss:2.609422
第17次训练的准确率为:84.81%
第18次训练 loss:2.536880
第18次训练的准确率为:85.6%
第19次训练 loss:2.439034
第19次训练的准确率为:87.1%
第20次训练 loss:2.386820
第20次训练的准确率为:87.05%
第21次训练 loss:1.739667
第21次训练的准确率为:89.16%
第22次训练 loss:1.591905
第22次训练的准确率为:89.22%
第23次训练 loss:1.531289
第23次训练的准确率为:89.15%
第24次训练 loss:1.495312
第24次训练的准确率为:89.17%
第25次训练 loss:1.453824
第25次训练的准确率为:89.7%
第26次训练 loss:1.421395
第